## Analyzing Short Term Listings Data With Python and Neo4j

In [ ]:
# Create a Neo4j driver
from neo4j.v1 import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "letmein"))

In [ ]:
# Hello world
with driver.session() as session:
    result = session.run("MATCH (n) RETURN COUNT(n);")
    for record in result:
        print(record)

## Pandas

We can use the popular [pandas](http://pandas.pydata.org/) with Neo4j. To create a DataFrame object from data from Neo4j simply pass the result object of a Cypher query into the DataFrame constructor.

In [ ]:
# !pip install pandas
from pandas import DataFrame

In [ ]:
with driver.session() as session:
    df = DataFrame(session.run('''
MATCH (n:Neighborhood)<-[:IS_IN]-(l:Listing)-[:HAS_AMENITY]->(a:Amenity) 
RETURN n.name AS neighborhood, l.name AS name, COLLECT(a.name) AS amenities, l.price AS price LIMIT 100
''').data())

In [ ]:
df

In [ ]:
# What are the most expensive neighborhoods?
with driver.session() as session:
    df = DataFrame(session.run('''
        MATCH (l:Listing)-[:IS_IN]->(n:Neighborhood)
        WITH n, avg(l.price) AS avg_price
        RETURN n.name AS neighborhood, avg_price ORDER BY avg_price DESC LIMIT 10
    ''').data())

df

## Exercise: What are the most expensive boroughs?

In [ ]:
# What are the most expensive boroughs?
# <INSERT CODE HERE>

## Plotting

We can use matplotlib to plot query results

In [ ]:
%matplotlib inline

In [ ]:
df.plot.bar(x="neighborhood")

## Exercise: Produce a plot of the most common amenities

In [ ]:
# What are the most common amenities?
# <INSERT CODE HERE>

## Natural Language Processing
### With TextBlob

We have review text ("comments") for each review. We can apply NLP techniques such as keyword extraction and sentiment analysis to enhance our data model. We will use the [Textblob Python toolkit](https://textblob.readthedocs.io/en/dev/)

In [ ]:
# !pip install textblob
# import nltk
# nltk.download('brown')
#nltk.download('punkt')
from textblob import TextBlob

In [ ]:
# fetch one review
with driver.session() as session:
    review = session.run("MATCH (r:Review) WHERE EXISTS(r.comments) RETURN r.comments LIMIT 1").single().value()
review

In [ ]:
# instantiate TextBlob object
blob = TextBlob(review)

# sentiment analysis
print ("Polarity: ")
print(blob.sentiment.polarity)
print()

# subjectivity
print("Subjectivity: ")
print(blob.sentiment.subjectivity)
print()

# keyword extraction (using noun phrases)
print ("Keywords: ")
print(blob.noun_phrases)

## Adding Sentiment To The Graph

In [ ]:
# calculate polarity and write back to the graph, Review node

with driver.session() as session:
    reviews = session.run('''
        MATCH (r:Review)
        WHERE EXISTS(r.comments) AND NOT EXISTS(r.polarity)
        RETURN r.comments AS review, r.review_id AS id LIMIT 2000
    ''')
    for record in reviews:
        blob = TextBlob(record['review'])
        polarity = blob.sentiment.polarity
        review_id = record['id']
        session.run('''
        MATCH (r:Review {review_id: $id})
        SET r.polarity = $polarity
        ''', id=review_id, polarity=polarity)

## Exercise: 

* Find the best and worst reviews by polarity.
* Find the listings with the best/worst average rating (by polarity).
* Find the host with the best/worst average rating (by polarity)

## Keyword extraction

We can extend the model to include keywords extracted from the reviews:

![](./images/datamodel_kw.png)

In [ ]:
# extract keywords and write back to graph

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (k:Keyword) ASSERT k.name IS UNIQUE;")
    reviews = session.run('''
        MATCH (r:Review)-[:REVIEWS]->(l:Listing) 
        WHERE EXISTS(r.comments) AND NOT EXISTS((l)-[:CONTAINS]->(:Keyword))
        RETURN r.comments AS review, l.listing_id AS id LIMIT 2000
    ''')
    for record in reviews:
        blob = TextBlob(record['review'])
        keywords = blob.noun_phrases
        review_id = record['id']
        session.run('''
        MATCH (l:Listing) WHERE l.listing_id = {listing_id}
        UNWIND {keywords} AS kw
        MERGE (k:Keyword {name: kw})
        MERGE (l)-[:CONTAINS]->(k)
        ''', listing_id=review_id, keywords=keywords)

## Exercise: Recommendations
Use our new review keywords to improve our previous recommendation queries

In [ ]:
# Improved recommendations using keywords
# <INSERT CODE HERE>